In [3]:
import glob
from textgrid import TextGrid, IntervalTier
import csv
import numpy as np

def addToDict(word, time,dictionary):
    if word in dictionary:
        dictionary[word].append(time)
    else:
        dictionary[word]=[time]


languageRootFolders = glob.glob('./common-voice-forced-alignments-main/*')
for language in languageRootFolders:
    alignments = glob.glob(language+"/alignments/**/*.TextGrid",recursive=True)
    alignments = filter(lambda x: x.endswith("TextGrid"), alignments)
    alignments=set(alignments)
    phoneDict={}
    wordDict={}
    for alignment in alignments:
        tg = TextGrid()
        try:
            tg.read(alignment)
            words=tg[0]
            for interval in words:
                time=interval.maxTime-interval.minTime
                word=interval.mark
                addToDict(word,time,wordDict)
            phones=tg[1]
            for interval in phones:
                time=interval.maxTime-interval.minTime
                phone=interval.mark
                addToDict(phone,time,phoneDict)
        except Exception:
            pass
    header = ["repetitions", "orthographic_form","meanDuration","medianDuration","stDevDuration"]
    
    with open(language+"-phone.csv", 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for elem in phoneDict:
            times=phoneDict[elem]
            repetitions=len(times)
            orthographic_form=elem
            meanDuration=np.mean(times)
            medianDuration=np.median(times)
            stDevDuration=np.std(times)
            if orthographic_form is None:
                orthographic_form=""
            writer.writerow([repetitions,orthographic_form,meanDuration,medianDuration,stDevDuration])
    f.close()
        
    with open(language+"-word.csv", 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for elem in wordDict:
            times=wordDict[elem]
            repetitions=len(times)
            orthographic_form=elem
            meanDuration=np.mean(times)
            medianDuration=np.median(times)
            stDevDuration=np.std(times)
            if orthographic_form is None:
                orthographic_form=""
            writer.writerow([repetitions,orthographic_form,meanDuration,medianDuration,stDevDuration])
    f.close()

    
